# Basics of Mobile Robotics: Project Report

Authors:
Louis Gevers, Paul Blossier, Zing
Shawn Tan, Naël Dillenbourg

## 1. Robot Environment

The robot has to navigate a square environment on a white sheet, where it has to reach a blue square while avoiding the red ones, from wherever it is released.
We make use of ArUco markers, whose detection is robust, fast, and simple [1].
Four markers are used to denote the corners of the environment, and a fifth one is put on the robot to obtain the relative position and orientation of the robot with respect to the corners.
We first used small colored squares, but as we will show in the vision section, we have to choose the correct values to detect these colours, which can change with different lighting conditions.
We therefore chose to only calibrate for the goal and obstacles, blue and red squares respectively, and not the corners and robot position, which use ArUco markers and therefore do not need adjustements with different lighting.

The environment for this project has two types of obstacles:
1. Permanent obstacles which are accounted for in the path planning and detected by our vision module (the red squares).
2. Temporary obstacles which we cannot account for during path planning. These are detected by the local navigation module that takes control when an object is close enough to the proximity sensors.

We chose a sparse, non-discrete environment, as we were more comfortable for the Thymio to do long straight lines with occasional small turns rather than the many rotations with small advancements required in a grid-like environment (it would also be faster).
As we went on with the project we discovered that this brings its challenges, as it is much harder to update the map when seeing obstacles and the control is not as straightforward.
If we were to do the project again we would probably go for a grid-like structure for easier map updates.

## 2. Project Structure

## 3. Implementation

## 4. Demo

## References

[1] https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html